In [1]:
import sugartensor as tf
from data import SpeechCorpus, voca_size
from model import *

tf.sg_verbosity(10)
batch_size = 16    # total batch size
# corpus input tensor
data = SpeechCorpus(batch_size=batch_size * tf.sg_gpus())

I 0709:23:19:47.559:data.py:147] TRAIN set loaded.(total data=13388, total batch=836)


In [2]:
# sentence label as inputs
inputs = tf.split(data.label.sg_float(), tf.sg_gpus(), axis=0)
# mfcc feature of audio as outputs
labels = tf.split(data.mfcc, tf.sg_gpus(), axis=0)

In [3]:
inputs, labels

([<tf.Tensor 'split:0' shape=(16, ?) dtype=float32>],
 [<tf.Tensor 'split_1:0' shape=(16, ?, 20) dtype=float32>])

In [ ]:
#tf.not_equal(inputs[0].sg_sum(axis=2), 0.).sg_int().sg_sum(axis=1).get_shape()

In [ ]:
#tf.not_equal(labels[0], 0).sg_int().sg_sum(axis=1).get_shape()

In [4]:
# sequence length except zero-padding
seq_len = []
for input_ in inputs:
    #seq_len.append(tf.not_equal(input_.sg_sum(axis=2), 0.).sg_int().sg_sum(axis=1))
    seq_len.append(tf.not_equal(input_, 0).sg_int().sg_sum(axis=1))

In [5]:
seq_len

[<tf.Tensor 'Sum:0' shape=(16,) dtype=int32>]

In [6]:
logit = get_logit(inputs[0].sg_expand_dims(), voca_size=voca_size)
logit.get_shape()

TensorShape([Dimension(16), Dimension(None), Dimension(20)])

### already performed: `sg_squeeze().sg_transpose(perm=[1, 2, 0])`

In [ ]:
logit.sg_mse(target=labels[0]).sg_sum(axis=2).sg_sum(axis=1).get_shape()

### $\mu$ __-  law?__

In [8]:
logit.sg_ctc(target=inputs[0], seq_len=seq_len).get_shape()

TypeError: Input 'labels_values' of 'CTCLoss' Op has type float32 that does not match expected type of int32.

In [ ]:
# parallel loss tower
@tf.sg_parallel
def get_loss(opt):
    # encode audio feature
    logit = get_logit(opt.input[opt.gpu_index], voca_size=voca_size)
    # CTC loss
    #return logit.sg_ctc(target=opt.target[opt.gpu_index], seq_len=opt.seq_len[opt.gpu_index])
    # MSE loss
    return logit.sg_mse(target=opt.target[opt.gpu_index]).sg_sum(axis=3).sg_sum(axis=2)

In [ ]:
#
# train
#
#loss=get_loss(input=inputs, target=label0, seq_len=seq_len)
loss=get_loss(input=inputs, target=labels)
tf.sg_train(lr=0.0001, loss=loss, ep_size=data.num_batch, max_ep=50)

In [ ]:
voca_size

In [ ]:
var1 = tf.Variable([[[2., -1., 3.], [3., 1., -2.]], [[1., -1., 2.], [3., 1., -2.]]],name='var1')

In [ ]:
var2 = tf.Variable([[2., 1.], [2., 3.]],name='var2')

In [ ]:
var1.get_shape(), var2.get_shape()

In [ ]:
var1.sg_ctc(target=var2)